In [1]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import HTML

from ipywidgets import (
    interactive_output, IntSlider, VBox, HBox, Checkbox, Dropdown, Label, Button
)
from IPython.display import display, HTML

import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
project_root = Path.cwd().parents[0]
sys.path.append(str(project_root))
from src.preference_factors.build_preference_dataset import (
    build_preference_factor_dataset
)

import src.analysis.basic_analytics as ba

## Monthly

In [2]:
df_monthly = build_preference_factor_dataset(
    returns_file="sp500_returns_monthly_with_tickers.csv",
    market_caps_file="sp500_market_caps_monthly.csv",
    ff_factors_file="ff_factors_monthly.csv",
    frequency="monthly",
)


In [3]:
df_monthly = df_monthly.rename(columns={
    "CW": "Cap-Weighted",
    "EW": "Equal-Weighted",
    "CW-EW": "Preference Portfolio: CW–EW"
})


In [4]:
# Define options
factor_options = ["MKT_RF", "SMB", "HML", "RMW", "CMA", "MOM"]
metric_options = ["Mean", "Std Dev", "r_squared", "alpha_tstat", "alpha_pval", "sortino"]
y_options = [
    "Cap-Weighted",                              
    "Equal-Weighted",
    "Preference Portfolio: CW–EW"
]


In [5]:
# Helper functions
def create_checkboxes(options, default=[]):
    return [Checkbox(value=(opt in default), description=opt, indent=False) for opt in options]

def get_selected_options(checkboxes):
    return [cb.description for cb in checkboxes if cb.value]

# Create widgets
factor_checkboxes = create_checkboxes(factor_options, default=["SMB", "HML", "RMW", "CMA", "MOM"])
metric_checkboxes = create_checkboxes(metric_options, default=["Mean", "Std Dev"])
y_selector = Dropdown(options=y_options, value="Preference Portfolio: CW–EW", description="Portfolio")
window_slider = IntSlider(min=6, max=60, step=6, value=36, description="Window")
run_button = Button(description="Run", button_style='success')

############# additional graphs with market comparison and rolling stats #############

compare_to_market_checkbox = Checkbox(value=True, description="Compare to Market (Cap-Weighted)", indent=False)

In [6]:
# Store output to update
output_area = widgets.Output()


In [7]:
# Define wrapped function for plotting
def wrapped_plot_rolling(button=None):  # optional param for button click
    with output_area:
        output_area.clear_output()
        factors = get_selected_options(factor_checkboxes)
        table_metrics = get_selected_options(metric_checkboxes)
        y_var = y_selector.value
        window = window_slider.value

        selected_df = ba.compute_rolling_betas_and_alpha(
            df_monthly,
            portfolio_col=y_var,
            window=window,
            beta_MKT="MKT_RF" in factors,
            beta_SMB="SMB" in factors,
            beta_HML="HML" in factors,
            beta_RMW="RMW" in factors,
            beta_CMA="CMA" in factors,
            beta_Momentum="MOM" in factors
        )
        
        market_df = ba.compute_rolling_betas_and_alpha(
            df_monthly,
            portfolio_col="Cap-Weighted", 
            window=window,
            beta_MKT="MKT_RF" in factors,
            beta_SMB="SMB" in factors,
            beta_HML="HML" in factors,
            beta_RMW="RMW" in factors,
            beta_CMA="CMA" in factors,
            beta_Momentum="MOM" in factors
            )

        # Plot rolling alpha and betas
        plot_cols = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]
        selected_df[plot_cols].plot(figsize=(12, 6), title=f"{window}-Month Rolling Alpha & Betas")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        ## compare to market
        if compare_to_market_checkbox.value:
            fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

            # Plot selected portfolio
            selected_df[plot_cols].plot(ax=axes[0], legend=False)
            axes[0].set_title(f"{y_var}")
            axes[0].grid(True)

            # Plot market portfolio
            market_df[plot_cols].plot(ax=axes[1], legend=False)
            axes[1].set_title("Cap-weighted (Market)")
            axes[1].grid(True)

            # Shared legend on top
            # After plotting on both axes:
            handles1, labels1 = axes[0].get_legend_handles_labels()
            handles2, labels2 = axes[1].get_legend_handles_labels()

            # Combine and deduplicate by label
            combined = dict(zip(labels1 + labels2, handles1 + handles2))

            # Title first
            fig.text(0.5, 1.12, "Selected Portfolio and Market Comparison", ha='center', fontsize=16, weight='bold')
            fig.suptitle(f"{window}-Month Rolling Alpha & Beta Comparison", fontsize=14, y=1.04)


            # Then a single legend with unique entries
            fig.legend(combined.values(), combined.keys(), loc='upper center', bbox_to_anchor=(0.5, 1.01), ncol=len(combined), fontsize=10)


            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.show()
            
            # more graphs for comparison
            # Get raw excess return series for selected portfolio and market
            sel_returns = df_monthly[y_var]
            mkt_returns = df_monthly["Cap-Weighted"]

            # Compute rolling metrics
            cum_sel, vol_sel, sharpe_sel = ba.compute_rolling_metrics(sel_returns, window)
            cum_mkt, vol_mkt, sharpe_mkt = ba.compute_rolling_metrics(mkt_returns, window)

            # Plot side-by-side pairs

            # --- Cumulative Returns ---
            fig, axes = plt.subplots(1, 2, figsize=(16, 4), sharey=True)
            axes[0].plot(cum_sel, label=y_var)
            axes[0].set_title(f"Cumulative Returns: {y_var}")
            axes[1].plot(cum_mkt, label="Cap-Weighted", color="orange")
            axes[1].set_title("Cumulative Returns: Market")
            for ax in axes:
                ax.grid(True)
                ax.set_ylabel("Log Return")
            plt.tight_layout()
            plt.show()

            # --- Rolling Volatility ---
            fig, axes = plt.subplots(1, 2, figsize=(16, 4), sharey=True)
            axes[0].plot(vol_sel, label=y_var)
            axes[0].set_title(f"{window}-Month Rolling Volatility (Annualized): {y_var}")
            axes[1].plot(vol_mkt, label="Cap-Weighted", color="orange")
            axes[1].set_title(f"{window}-Month Rolling Volatility (Annualized): Market")
            for ax in axes:
                ax.grid(True)
                ax.set_ylabel("Volatility")
            plt.tight_layout()
            plt.show()

            # --- Rolling Sharpe Ratio ---
            fig, axes = plt.subplots(1, 2, figsize=(16, 4), sharey=True)
            axes[0].plot(sharpe_sel, label=y_var)
            axes[0].set_title(f"{window}-Month Rolling Sharpe Ratio (RF=0): {y_var}")
            axes[1].plot(sharpe_mkt, label="Cap-Weighted", color="orange")
            axes[1].set_title(f"{window}-Month Rolling Sharpe Ratio (RF=0): Market")
            for ax in axes:
                ax.grid(True)
                ax.set_ylabel("Sharpe Ratio")
            plt.tight_layout()
            plt.show()



        # Summary table
        all_metrics = {
            "Mean": selected_df.mean(),
            "Std Dev": selected_df.std(),
            "r_squared": selected_df["r_squared"].mean() if "r_squared" in selected_df.columns else None,
            "alpha_tstat": selected_df["alpha_tstat"].mean() if "alpha_tstat" in selected_df.columns else None,
            "alpha_pval": selected_df["alpha_pval"].mean() if "alpha_pval" in selected_df.columns else None,
            "sortino": selected_df["sortino"].mean() if "sortino" in selected_df.columns else None
        }

        selected_metrics = {k: v for k, v in all_metrics.items() if k in table_metrics and v is not None}
        row_names = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]

        summary = pd.DataFrame({metric: selected_df[row_names].mean() if metric == "Mean"
                                else selected_df[row_names].std() if metric == "Std Dev"
                                else [selected_metrics[metric]] * len(row_names)
                                for metric in selected_metrics})

        summary.index = row_names
        summary = summary.round(4)

        global latest_summary_table
        latest_summary_table = summary

        #display(HTML("<h4>Average Results from Rolling Factor Model</h4>"))
        display(widgets.HTML("<h4>Average Results from Rolling Factor Model</h4>"))

        display(summary)


In [8]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox

# Bind the run button
run_button.on_click(wrapped_plot_rolling)

ui = VBox([
    HBox([Label("Factors:"), VBox(factor_checkboxes)]),
    HBox([Label("Summary Columns:"), VBox(metric_checkboxes)]),
    y_selector,
    compare_to_market_checkbox,
    window_slider,
    run_button,
    output_area
])

title_html = widgets.HTML(
    "<h2 style='color:#003366;'>Rolling Factor Analysis App – Preference Portfolio Variants</h2>"
)

instructions_html = widgets.HTML("""
<p><b>Instructions:</b> Select a portfolio (e.g., Cap-weighted, Equal-weighted, or a preference variant), 
choose the factors to include, set the rolling window, and click <b>Run</b> to generate plots and summary statistics.</p>
<p>Explore alpha, beta exposures, Sortino ratio, and R² over time.</p>
""")

display(VBox([title_html, instructions_html, ui]))
